In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import BertTokenizer
from loralib.layers import Linear
from loralib.utils import mark_only_lora_as_trainable

upload dataset

In [5]:
dataset = load_dataset("ag_news")
print(dataset["train"][0])

Using the latest cached version of the dataset since ag_news couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\Garry\.cache\huggingface\datasets\ag_news\default\0.0.0\eb185aade064a813bc0b7f42de02595523103ca4 (last modified on Sat Aug 16 16:10:13 2025).


{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}


BERT pre-training

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # BERT分词器分词

def encode_with_bert(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding='max_length',
        max_length=128,
    )
    return {
        "input_ids": tokens["input_ids"],
        "attention_mask": tokens["attention_mask"],
        "label": example["label"]
    }

encoded_dataset = dataset.map(encode_with_bert, batched=True) # Hugging Face Datasets库的map函数批量处理数据

encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"]) # 设置数据格式为PyTorch张量，并指定需要的列


Prepare DataLoader

In [7]:
train_loader = DataLoader(encoded_dataset["train"], batch_size=32, shuffle=True)
test_loader = DataLoader(encoded_dataset["test"], batch_size=32)

Define RNN model with LoRA classification head

In [8]:
class LoRARNNCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.i2h = Linear(input_size, hidden_size, r=8)
        self.h2h = Linear(hidden_size, hidden_size, r=8)

    def forward(self, x, h):
        return torch.tanh(self.i2h(x) + self.h2h(h))

class TextRNNWithLoRA(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn_cell = LoRARNNCell(embed_dim, hidden_dim)
        self.hidden_dim = hidden_dim
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids):
        embedded = self.embedding(input_ids)
        batch_size, seq_len, _ = embedded.size()
        h_t = torch.zeros(batch_size, self.hidden_dim, device=embedded.device)

        for t in range(seq_len):
            h_t = self.rnn_cell(embedded[:, t, :], h_t)

        return self.fc(h_t)


In [9]:
# Initialize model
model = TextRNNWithLoRA(vocab_size=tokenizer.vocab_size, embed_dim=128, hidden_dim=128, num_classes=4)
# Load pretrained weights from IMDB except fc
state_dict = torch.load("rnn_imdb.pth")
filtered_dict = {k: v for k, v in state_dict.items() if not k.startswith("fc")}
model.load_state_dict(filtered_dict, strict=False)

# freeze all except LoRA
mark_only_lora_as_trainable(model)

# Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and loss
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
criterion = nn.CrossEntropyLoss()


Train model

In [10]:
for epoch in range(20):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1, Loss: 1.3767
Epoch 2, Loss: 1.3739
Epoch 3, Loss: 1.3739
Epoch 4, Loss: 1.3735
Epoch 5, Loss: 1.3737
Epoch 6, Loss: 1.3740
Epoch 7, Loss: 1.3737
Epoch 8, Loss: 1.3755
Epoch 9, Loss: 1.3750
Epoch 10, Loss: 1.3749
Epoch 11, Loss: 1.3749
Epoch 12, Loss: 1.3756
Epoch 13, Loss: 1.3771
Epoch 14, Loss: 1.3777
Epoch 15, Loss: 1.3765
Epoch 16, Loss: 1.3757
Epoch 17, Loss: 1.3754
Epoch 18, Loss: 1.3753
Epoch 19, Loss: 1.3762
Epoch 20, Loss: 1.3773


Evaluation

In [11]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids)
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {correct / total:.2%}")



Test Accuracy: 28.46%
